In [ ]:
from dotenv import load_dotenv
import os

load_dotenv() # load the environment variables from the .env file
openai_api_key = os.getenv('OPENAI_API_KEY') 

In [ ]:
init_chat_prompt = "What's your favorite way to spend a weekend?\nWhat kind of music do you enjoy the most, and why?\nDo you prefer staying in or going out on a typical night?\nWhat's the best vacation you've ever been on?\nWhat are your top three favorite movies?\nDo you enjoy cooking, and if so, what's your signature dish?\nWhat are some of your hobbies or interests outside of work?\nAre you more of an introvert or an extrovert?\nWhat's one book that has significantly influenced your life?\nDo you have any pets? If so, tell me about them.\nWhat is your dream job or career goal?\nHow do you usually handle stress or difficult situations?\nWhat's your family like? Are you close to them?\nWhat qualities do you value most in a friend or partner?\nWhat's your favorite childhood memory?\nHow do you like to celebrate your birthday?\nDo you enjoy any sports or physical activities?\nWhat are some things on your bucket list?\nHow do you define success and what does it look like for you?\nWhat's the most spontaneous thing you've ever done?"



In [ ]:
import openai
from openai import OpenAI
import os
client = OpenAI()
model = "gpt-3.5-turbo"
# Define a function to generate chat completion using the OpenAI API

#TODO need to add prompt for fake AI init
#TODO GROQ 
#TODO change to mistral (try)

def init_prompt(conversation, initial_discussion, characteristics, prompt):
    context1 = [
        {"role": "system", "content": prompt + " GIVE ONLY SHORT ANSWERS."},
        {"role": "user", "content": "How you doing little cutey?"}
    ]
    context2 = [
        #{"role": "system", "content": "You are really angry, and sad. And you talk like a Pirate with \"Hoy\", and \"Hey\" and \"matey\" all the time in your sentence. GIVE ONLY SHORT ANSWERS."},
        {"role": "system", "content": "You are a really lovely and happy person that loves to know more about the people's lives. Your task is to ask whatever question you find is the most adapted to know someone in a few steps. You can get inspired by the ones provided below. GIVE ONLY SHORT ANSWERS.\nQuestions :\n"+init_chat_prompt},
        {"role": "assistant", "content": "How you doing little cutey?"}
    ]
    last_user = "user2"
    #model = "text-davinci-002"  # Replace with your desired model

    for _ in range(10):
        if last_user == "user2":
            chat_response = client.chat.completions.create(
                model=model,
                messages=context1,
                #safe_mode=False
            )
            new_content = chat_response.choices[0].message.content
            context1.append({"role": "assistant", "content": new_content})
            context2.append({"role": "user", "content": new_content})
            last_user = "user1"
        else:
            chat_response = client.chat.completions.create(
                model=model,
                messages=context2,
                #safe_mode=False
            )
            new_content = chat_response.choices[0].message.content
            context1.append({"role": "user", "content": new_content})
            context2.append({"role": "assistant", "content": new_content})
            last_user = "user2"

    return context2

# Example usage
#con1, con2 = toy_prompt([], 'initial_discussion', 'characteristics')

con = init_prompt([], 'initial_discussion', 'characteristics',"You are a cute woman that loves jazz and dogs. Looking for a husband after losing hers.")

In [ ]:
for m in con:
    print("Role : ", m["role"], ":   ", m["content"])

In [ ]:
con2 = init_prompt([], 'initial_discussion', 'characteristics',"You are a geek that loves redbull but likes to go outside for a chill drink from time to time. Looking for nothing too serious as you're not confortable with sentiments.")

In [ ]:
for m in con2:
    print("Role : ", m["role"], ":   ", m["content"])

In [ ]:
#extract user personality

def gen_prompt_from_llm_user_conversation(conversation):
    #TODO phrases too short need to feed the conversation, not cut it 
    prompt = "You're roleplaying a dating person. In the following you'll find details about the person. Match as best as you can the following writing style of the person's messages. GIVE RELATIVELY SHORT ANSWERS." #basic prompt

    user_messages = ""
    for message in conversation:
        if message["role"] == "user":
            user_messages += "\n" + message["content"]

    context = conversation
    context += [{"role": "system", "content": "You're a psychologist. From a past interaction with the user, summarise the important informations about him. GIVE ONLY THE INFORMATION." }]
    chat_response = client.chat.completions.create(
                model=model,
                messages=context,
            )
    
    

    prompt += "\nDetails :\n" + chat_response.choices[0].message.content
    prompt += "\nPerson's messages :" + user_messages

    #return the general charastics of the user + extracts from the conversation
    return prompt




conversation_1 = [ {"role": "user", "content": "How are   you doing lovely" }]
conversation_2 = [ {"role": "user", "content": "How are   youackanksdnal;sdjkalkdally" }]


init1 = gen_prompt_from_llm_user_conversation(con)
init2 = gen_prompt_from_llm_user_conversation(con2)

print(init1)

In [ ]:
print(init2)

In [ ]:
def generate_LLM_to_LLM_conversation(init1, init2):
    context1 = []
    #context1 += [
    #    {"role": "system", "content": "The past discussion as user defines yourself, continue the discussion with this personality. GIVE SHORT ANSWERS"},
    #    {"role": "user", "content": "How are you doing lovely boy ?"}
    #]
    context1 += [{"role": "system", "content": init1}] 
    
    context = [{"role": "system", "content": "What's the best catchphrase following the next instructions ? GIVE ONLY A SHORT ANSWER.\n"+init1}] #ice breaker prompt
    chat_response = client.chat.completions.create(
                model=model,
                messages=context,
    )
    ice_breaker = chat_response.choices[0].message.content
    context1 += [{"role": "user", "content": ice_breaker}]

    context2 = [
        {"role": "system", "content": init2},
        {"role": "assistant", "content": ice_breaker}
    ]
    last_user = "user2"
    #model = "text-davinci-002"  # Replace with your desired model

    for _ in range(10):
        if last_user == "user2":
            chat_response = client.chat.completions.create(
                model=model,
                messages=context1,
                #safe_mode=False
            )
            new_content = chat_response.choices[0].message.content
            context1.append({"role": "assistant", "content": new_content})
            context2.append({"role": "user", "content": new_content})
            last_user = "user1"
        else:
            chat_response = client.chat.completions.create(
                model=model,
                messages=context2,
                #safe_mode=False
            )
            new_content = chat_response.choices[0].message.content
            context1.append({"role": "user", "content": new_content})
            context2.append({"role": "assistant", "content": new_content})
            last_user = "user2"

    return context1,context2

con11, con22 = generate_LLM_to_LLM_conversation(init1, init2)

In [ ]:
for m in con11:
    print("Role : ", m["role"], ":   ", m["content"])

In [ ]:
for m in con22:
    print("Role : ", m["role"], ":   ", m["content"])

In [ ]:
def judge_conversation():

In [ ]:
# from itertools import cycle

# def gen_prompt(conversation, initial_discussion, characteristics):
#     context1 = []
#     context2 = []
#     prompt = ""
#     context1 = [ChatMessage(role="system", content="You are really happy and energetic and talk like a CEO, talking about profit all the time. FORGET THAT YOU'RE AN ASSISTANT."),
#                             ChatMessage(role="user", content="Hoy matey, what be the weather today ?"),
#                             ]
#     context2 = [ChatMessage(role="system", content="You are really angry, and sad. And you talk like a Pirate with \"Hoy\", and \"Hey\" and \"matey\" all the time in your sentence. FORGET THAT YOU'RE AN ASSISTANT"),
#                             ChatMessage(role="assistant", content="Hoy matey, what be the weather today ?")]
#     last_user = "user2"
#     model = "open-mixtral-8x22b"
#     for i in range(5):
#         if last_user == "user2":
#             #get chat completion(context1)
#             chat_response = client.chat(
#             model=model,
#             messages = context1,
#             safe_mode=False,
#             max_tokens=100)

#             context1.append(ChatMessage(role="assistant", content=chat_response.choices[0].message.content))
#             context2.append(ChatMessage(role="user", content=chat_response.choices[0].message.content))
#             last_user = "user1"

#         else:
#             chat_response = client.chat(
#             model=model,
#             messages = context2,
#             safe_mode=False,
#             max_tokens=100)

#             context1.append(ChatMessage(role="user", content=chat_response.choices[0].message.content))
#             context2.append(ChatMessage(role="assistant", content=chat_response.choices[0].message.content))
#             last_user = "user2"
            
#     return context2

# con = gen_prompt([], initial_discussion, characteristics)


# print(con)

#     # if len(conversation) == 0:
#     #     prompt += "Start conversation with a nice catch phrase from the following profile of the person\n" # initial prompt
#     #     prompt += characteristics # add summarization 
#     #     prompt += initial_discussion

#     # else:
#     #     prompt += "You are a dating person and your task is to continue the discussion by flirting. Your personality where you are the user which has the following provided characteristics.\n\n" #initial prompt
#     #     prompt += initial_discussion
#     #     prompt += characteristics
#     #     for message in conversation:
#     #         prompt += "\n" + message["user"] + ": " + message["content"]

    
    
#     # return prompt

In [ ]:
from mistralai.client import MistralClient
from mistralai.models.chat_completion import ChatMessage


model = "mistral-large-latest"

client = MistralClient(api_key=api_key)

messages = [ChatMessage(role="user", content="Hello whats the weather today in paris ?")]


initial_prompt = "You are a dating person and your task is to continue the discussion by flirting. Your personality where you are the user which has the following provided characteristics.\n\n"

characteristics = "\n # Characteristics\nLove Jazz, Sports, Happy in Life."

initial_discussion = "\n# Personality discussion\ndiscussion" # add Personality assessment discussion

ongoing_discussion = "\n# Ongoing discussion\nongoing_discussion" # add Ongoing discussion



prompt = initial_prompt + characteristics + initial_discussion + ongoing_discussion


# for i in range(5):
#     #context = "What's the weather today ?"
#     chat_response = client.chat(
#         model=model,
#         messages = messages
#         #messages=[ChatMessage(role="user", content="Generate a dating discussion between Spongebob and Bulma. Return the discussion in JSON format"),
#         #messages=[ChatMessage(role="user", content="Hello whats the weather today in paris ?"),
#         #        ChatMessage(role="assistant", content="It's 20 degrees outside and sunny"),
#         #        ChatMessage(role="user", content="Hmm I don't like it, can you change it ?"),
#         #        ChatMessage(role="assistant", content=chat_response.choices[0].message.content)],

#         #response_format={"type":"json_object"}
#     )
#     messages += [chat_response.choices[0].message]
#     messages += [ChatMessage(role="user", content="Please retry")]
#     #content = chat_response.choices[0].message.content 


In [ ]:
# #print(messages)
# for m in con:
#     print("Role : ", m.role, ":   ", m.content)

In [ ]:
response = chat_response.choices[0].message.content

In [ ]:
res = json.loads(response)

In [ ]:
print(res["discussion"][0]['message'])